In [257]:
import pandas as pd 
import numpy as np

df = pd.read_excel(r"C:\Users\wallace.souza\Downloads\LOGISTICA 2.0 - CONTROLE FINANCEIRO DE LOGÍSTICA.xlsx")
fn = pd.read_excel(r"C:\Users\wallace.souza\Downloads\Fornecedores.xlsx")


In [258]:
df = df.merge(fn, on='Código Fornecedor:', how='left')

In [259]:
df = df[df['Hora de início'].notnull()]

In [260]:
pd.set_option('display.max_columns', None)


In [261]:
df = df[['Hora de conclusão','Setor:\n',
       'Total de Pallets:','Total de Volumes:',
       'Filmagem de Quantos Palets ?','Total de Palets MISTURADOS?','Lona Truck ?','Lona Carreta ?',
       'Total $ Recebido em CASH?', 'Total Recebido por PIX ?',
       'Valor total dos Serviços ?', 'Valor Pago:', 'Valor da Diferença:',
      'Código Fornecedor:','Nome do Fornecedor:_x','Nome do Fornecedor:_y', 'Nome da Transportadora:',    
       'Data do Recebimento dos Valores:', 'Data do Serviço:',
       'Constava agendamento para hoje:',
        'Reg']]

df = df.rename(columns={'Nome do Fornecedor:_x':'Nome do Fornecedor: - Ivo'})
df = df.rename(columns={'Nome do Fornecedor:_y':'Nome do Fornecedor:'})

In [262]:
df['Valor Pago:'] = pd.to_numeric(df['Valor Pago:'], errors='coerce')


In [263]:
df['Valor total dos Serviços ?'] = pd.to_numeric(df['Valor total dos Serviços ?'], errors='coerce')


In [264]:
df[df['Valor Pago:'].isna()]
df['Nome do Fornecedor:'] = df['Nome do Fornecedor:'].str.strip()

In [265]:
df['%'] = df['Valor Pago:'] / df['Valor total dos Serviços ?'] * 100 - 100
df['Taxa'] = np.where(df['Total Recebido por PIX ?'] != 0, 'Taxa Pix?', "Ok")
df['Total de Pallets:'] = df['Total de Pallets:'].astype(float)
df['Total de Volumes:'] = df['Total de Volumes:'].astype(float)
df['Filmagem de Quantos Palets ?'] = df['Filmagem de Quantos Palets ?'].astype(float)

peso_pallet = np.where(df['Setor:\n'] == 'Perecíveis', 30, 25)
fornecedor = np.where(df['Nome do Fornecedor:'] == 'AMBEV S/A',20, peso_pallet)
lona = np.where(df['Lona Carreta ?'] == 'Sim',70, 0)
deslonamento = np.where(df['Lona Truck ?'] == 'Sim',50, 0)

df['Total de Volumes:']
	

denominador = (
        df['Total de Pallets:'] * fornecedor +
        df['Total de Volumes:'] + deslonamento +
        df['Filmagem de Quantos Palets ?'] * 20 +
        df['Total de Palets MISTURADOS?'] * 60 + lona
        
    )
    
# Cálculo da coluna final
df['Valor Pallet'] = df['Valor total dos Serviços ?'] / denominador
df['Valor Correto'] = denominador


In [266]:
colunas = ['Total $ Recebido em CASH?', 'Total Recebido por PIX ?',
           'Valor total dos Serviços ?', 'Valor Pago:', 'Valor da Diferença:']

df[colunas] = df[colunas].apply(
    lambda col: pd.to_numeric(col, errors='coerce')
)


In [267]:
df['Diferença'] = df['Valor total dos Serviços ?'] - df['Valor Correto']
df['Pix + Cash'] = (df['Total $ Recebido em CASH?'] + df['Total Recebido por PIX ?']) / df['Valor Pago:']


In [268]:
df['Data'] = df['Data do Recebimento dos Valores:'].dt.month

In [269]:
df1 = df[df['Data']== 12]

In [270]:
df1['Reg'] = df1['Reg'].astype(int) 
df1['Código Fornecedor:'] = df1['Código Fornecedor:'].astype(int) 

C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_24196\2778285773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Reg'] = df1['Reg'].astype(int)
C:\Users\wallace.souza\AppData\Local\Temp\ipykernel_24196\2778285773.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Código Fornecedor:'] = df1['Código Fornecedor:'].astype(int)


In [271]:
df1[(df1['Valor Pallet'] != 1) | (df1['Pix + Cash'] != 1) | (df1["Nome do Fornecedor:"].isna())]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data


In [272]:
noshow = df[(df['Constava agendamento para hoje:']=='Sim, No Show pendente') | (df['%']>=30)]
nao = df[(df['Constava agendamento para hoje:'].str.contains('não', na=False, case=False))]

In [273]:
print('pagamento em pix:' ,len(df[df['Total Recebido por PIX ?'] !=0]))
print('pagamento em dinheiro:' , len(df[df['Total Recebido por PIX ?'] ==0]))

pagamento em pix: 1648
pagamento em dinheiro: 1521


In [274]:
print('noshow = pagamento em pix:', len(noshow[noshow['Total Recebido por PIX ?'] !=0]))
print('noshow = pagamento em dinheiro:' , len(noshow[noshow['Total Recebido por PIX ?'] ==0]))

noshow = pagamento em pix: 252
noshow = pagamento em dinheiro: 54


In [275]:
print('não agendado = pagamento em pix:', len(nao[nao['Total Recebido por PIX ?'] !=0]))
print('não agendado = pagamento em dinheiro:' , len(nao[nao['Total Recebido por PIX ?'] ==0]))

não agendado = pagamento em pix: 127
não agendado = pagamento em dinheiro: 60


In [276]:
df1[df1['Data do Serviço:'] != df1['Data do Recebimento dos Valores:']]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data
19201,2025-12-01 17:02:24,Mercearia,16.0,0.0,0.0,0.0,Não,Não,0.0,484.00,400.0,484.00,0.0,2325425,PACKPECK,PACKPECK DISTRIBUIDORA LTDA,PROPRIO,2025-12-01,2025-11-28,Sim,19203,21.0,Taxa Pix?,1.0,400.0,0.0,1.0,12
19230,2025-12-02 16:10:28,Mercearia,23.0,0.0,0.0,0.0,Não,Não,0.0,695.75,575.0,695.75,0.0,1367,MAMÃO,MAMAO DISTRIBUIDORA DE ALIMENTOS LTDA,PROPRIA,2025-12-02,2025-12-01,Sim,19232,21.0,Taxa Pix?,1.0,575.0,0.0,1.0,12


In [277]:
b = df1[(df1['Taxa'] == 'Taxa Pix?') & (df1['%'] == 0)]

In [278]:
b["Nome do Fornecedor:"].value_counts().reset_index()

,Nome do Fornecedor:,count
0,BRF S.A.,1


In [281]:
df1[(df1["Data do Recebimento dos Valores:"].dt.day == 2)]

,Hora de conclusão,Setor:\n,Total de Pallets:,Total de Volumes:,Filmagem de Quantos Palets ?,Total de Palets MISTURADOS?,Lona Truck ?,Lona Carreta ?,Total $ Recebido em CASH?,Total Recebido por PIX ?,Valor total dos Serviços ?,Valor Pago:,Valor da Diferença:,Código Fornecedor:,Nome do Fornecedor: - Ivo,Nome do Fornecedor:,Nome da Transportadora:,Data do Recebimento dos Valores:,Data do Serviço:,Constava agendamento para hoje:,Reg,%,Taxa,Valor Pallet,Valor Correto,Diferença,Pix + Cash,Data
19216,2025-12-02 10:46:37,Mercearia,7.0,28.0,4.0,0.0,Não,Não,0.0,404.69,283.0,404.69,0.0,11598,PEPSICO,PEPSICO DO BRASIL LTDA,PROPRIO,2025-12-02,2025-12-02,"Sim, No Show pendente",19218,43.0,Taxa Pix?,1.0,283.0,0.0,1.0,12
19217,2025-12-02 11:01:35,Mercearia,2.0,0.0,0.0,0.0,Não,Não,65.0,0.00,50.0,65.00,0.0,1565,BELA VISTA,LATICINIOS BELA VISTA LTDA,PROPRIA,2025-12-02,2025-12-02,"Sim, No Show pendente",19219,30.0,Ok,1.0,50.0,0.0,1.0,12
19218,2025-12-02 11:03:38,Mercearia,16.0,0.0,0.0,0.0,Não,Não,0.0,440.00,400.0,440.00,0.0,1112,EBD,EMPRESA BRASILEIRA DE DISTRIBUICAO LTDA,PROPRIO,2025-12-02,2025-12-02,Sim,19220,10.0,Taxa Pix?,1.0,400.0,0.0,1.0,12
19219,2025-12-02 11:06:04,Mercearia,2.0,0.0,0.0,0.0,Não,Não,50.0,0.00,50.0,50.00,0.0,1494,BARANO,BARANO ALIMENTOS LTDA,PROPRIO,2025-12-02,2025-12-02,Sim,19221,0.0,Ok,1.0,50.0,0.0,1.0,12
19220,2025-12-02 11:07:43,Mercearia,2.0,0.0,0.0,0.0,Não,Não,50.0,0.00,50.0,50.00,0.0,1708972,NORTE SALEIRA,NORTE SALINEIRA S/A INDUSTRIA E COMERCIO - NORSAL,PROPRIO,2025-12-02,2025-12-02,Sim,19222,0.0,Ok,1.0,50.0,0.0,1.0,12
19221,2025-12-02 11:10:28,Mercearia,2.0,0.0,0.0,0.0,Não,Não,0.0,55.00,50.0,55.00,0.0,1195,JRE,JRE COMERCIO LTDA,PROPRIO,2025-12-02,2025-12-02,Sim,19223,10.0,Taxa Pix?,1.0,50.0,0.0,1.0,12
19222,2025-12-02 11:21:21,Perecíveis,3.0,0.0,0.0,0.0,Não,Não,90.0,0.00,90.0,90.00,0.0,2048506,MRC FOODS,MRC FOODS DISTRIBUIDORA DE ALIMENTOS LTDA,PROPRIO,2025-12-02,2025-12-02,Sim,19224,0.0,Ok,1.0,90.0,0.0,1.0,12
19223,2025-12-02 11:59:36,Mercearia,5.0,0.0,0.0,0.0,Não,Não,125.0,0.00,125.0,125.00,0.0,1065,PARATI,PARATI INDUSTRIA E COMERCIO DE ALIMENTOS LTDA,PROPRIO,2025-12-02,2025-12-02,Sim,19225,0.0,Ok,1.0,125.0,0.0,1.0,12
19224,2025-12-02 12:04:30,Perecíveis,2.0,0.0,0.0,0.0,Não,Não,60.0,0.00,60.0,60.00,0.0,11609,FAZENDA AGUA LINDAS,FAZENDA AGUAS LINDAS INDUSTRIA E COMERCIO DE C...,PROPRIO,2025-12-02,2025-12-02,Sim,19226,0.0,Ok,1.0,60.0,0.0,1.0,12
19225,2025-12-02 12:06:25,Perecíveis,1.0,0.0,0.0,0.0,Não,Não,0.0,33.00,30.0,33.00,0.0,1838,MON CHOW,MON CHOU COMERCIO IMPORTACAO E EXPORTACAO LTDA,PROPRIO,2025-12-02,2025-12-02,Sim,19227,10.0,Taxa Pix?,1.0,30.0,0.0,1.0,12
